In [1]:
import collections
import json
import re
from itertools import chain
from multiprocessing import Pool
from pathlib import Path
from typing import *

import dotenv
import git
import pandas as pd
import pydriller
from pymongo import MongoClient

In [2]:
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
DATA_DIR, DATA_DIR.exists()

(PosixPath('*/bachelor-thesis/data'), False)

In [3]:
client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio
patch_urls_mongo_collection = db.patchUrls
df = pd.DataFrame(list(patch_urls_mongo_collection.find()))
df

,_id,Id@,VulnUrl@,PatchUrls@,VulnUrl,PatchUrls
0,6556d499917e89644f5900d0,6556d499917e89644f5900d0,https://security.snyk.io/vuln/SNYK-JAVA-ORGOJA...,[],https://security.snyk.io/vuln/SNYK-JAVA-ORGOJA...,[]
1,6556d49a917e89644f5900da,6556d49a917e89644f5900da,https://security.snyk.io/vuln/SNYK-JAVA-ORGPOS...,[https://github.com/pgjdbc/pgjdbc/commit/0afaa...,https://security.snyk.io/vuln/SNYK-JAVA-ORGPOS...,[https://github.com/pgjdbc/pgjdbc/commit/0afaa...
2,6556d49b917e89644f5900e3,6556d49b917e89644f5900e3,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
3,6556d49c917e89644f5900e5,6556d49c917e89644f5900e5,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
4,6556d49c917e89644f5900e6,6556d49c917e89644f5900e6,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
...,...,...,...,...,...,...
673,6556d4ab917e89644f590156,6556d4ab917e89644f590156,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[],https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[]
674,6556d4ac917e89644f590158,6556d4ac917e89644f590158,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[],https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[]
675,6556d4ac917e89644f59015a,6556d4ac917e89644f59015a,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[],https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[]
676,6556d4ac917e89644f59015c,6556d4ac917e89644f59015c,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...


In [4]:
df = df[df['PatchUrls'].map(len) > 0]
df

,_id,Id@,VulnUrl@,PatchUrls@,VulnUrl,PatchUrls
1,6556d49a917e89644f5900da,6556d49a917e89644f5900da,https://security.snyk.io/vuln/SNYK-JAVA-ORGPOS...,[https://github.com/pgjdbc/pgjdbc/commit/0afaa...,https://security.snyk.io/vuln/SNYK-JAVA-ORGPOS...,[https://github.com/pgjdbc/pgjdbc/commit/0afaa...
2,6556d49b917e89644f5900e3,6556d49b917e89644f5900e3,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
3,6556d49c917e89644f5900e5,6556d49c917e89644f5900e5,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
4,6556d49c917e89644f5900e6,6556d49c917e89644f5900e6,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
5,6556d49c917e89644f5900e7,6556d49c917e89644f5900e7,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...,https://security.snyk.io/vuln/SNYK-JAVA-ORGSPR...,[https://github.com/spring-projects/spring-fra...
...,...,...,...,...,...,...
670,6556d4ab917e89644f590153,6556d4ab917e89644f590153,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...
671,6556d4ab917e89644f590154,6556d4ab917e89644f590154,https://security.snyk.io/vuln/SNYK-JAVA-ORGXML...,[https://github.com/SvenEwald/xmlbeam/commit/f...,https://security.snyk.io/vuln/SNYK-JAVA-ORGXML...,[https://github.com/SvenEwald/xmlbeam/commit/f...
672,6556d4ab917e89644f590155,6556d4ab917e89644f590155,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...
676,6556d4ac917e89644f59015c,6556d4ac917e89644f59015c,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...


In [5]:
df_dict = df.to_dict(orient='records')
df_dict

[{'_id': ObjectId('6556d49a917e89644f5900da'),
  'Id@': ObjectId('6556d49a917e89644f5900da'),
  'VulnUrl@': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGPOSTGRESQL-2970521',
  'PatchUrls@': ['https://github.com/pgjdbc/pgjdbc/commit/0afaa71d5254804f8bab7caab96735c5f7d11aa1',
   'https://github.com/pgjdbc/pgjdbc/commit/739e599d52ad80f8dcd6efedc6157859b1a9d637',
   'https://github.com/pgjdbc/pgjdbc/commit/b5ee575b7d6e0918a58fe533f6c50b0c7e9f73c0'],
  'VulnUrl': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGPOSTGRESQL-2970521',
  'PatchUrls': ['https://github.com/pgjdbc/pgjdbc/commit/0afaa71d5254804f8bab7caab96735c5f7d11aa1',
   'https://github.com/pgjdbc/pgjdbc/commit/739e599d52ad80f8dcd6efedc6157859b1a9d637',
   'https://github.com/pgjdbc/pgjdbc/commit/b5ee575b7d6e0918a58fe533f6c50b0c7e9f73c0']},
 {'_id': ObjectId('6556d49b917e89644f5900e3'),
  'Id@': ObjectId('6556d49b917e89644f5900e3'),
  'VulnUrl@': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGSPRINGFRAMEWORK-1296829',
  'PatchUrls@': ['h

In [6]:
# patch_url -> vuln_url
patch_urls = collections.defaultdict(set)
for r in df_dict:
    for pu in r['PatchUrls']:
        patch_urls[pu].add(r['VulnUrl'])
len(patch_urls.keys())

484

In [7]:
def get_repo_and_commit(commit_url: str) -> Tuple[str, str]:
    for m in re.finditer(r"(?P<repo>(https://)?(www\.)?github\.com(?:/[^/]+)*)/commit/(?P<hash>[0-9a-f]+)(\#diff.*)?",
                         commit_url):
        return m.group('repo'), m.group('hash')

In [8]:
repo_patches = collections.defaultdict(set)
repo_commit_to_url = dict()
for p in patch_urls.keys():
    repo, commit = get_repo_and_commit(p)
    repo_patches[repo].add(commit)
    repo_commit_to_url[(repo, commit)] = p

repo_patches, len(repo_patches.keys()), len(list(chain(*repo_patches.values())))

(defaultdict(set,
             {'https://github.com/pgjdbc/pgjdbc': {'0afaa71d5254804f8bab7caab96735c5f7d11aa1',
               '14b62aca4764d496813f55a43d050b017e01eb65',
               '34855cffb9079edf95cf77930f3e3b73f6a1b139',
               '739e599d52ad80f8dcd6efedc6157859b1a9d637',
               '8a363a7c0989ef8a8f45bb055b4003f758ceabd5',
               '9008dc9aade6dbfe4efafcd6872ebc55f4699cf5',
               '91b8c2ca600f82d2ec47c3fb54ca2036b000e320',
               'b5ee575b7d6e0918a58fe533f6c50b0c7e9f73c0',
               'cdeeaca47dc3bc6f727c79a582c9e4123099526e',
               'e73c6b6b53ff11bc0bd06c44130a880d3bf77c55',
               'f4d0ed69c0b3aae8531d83d6af4c57f22312c813',
               'f6d47034a4ce292e1a659fa00963f6f713117064'},
              'https://github.com/spring-projects/spring-framework': {'002546b3e4b8d791ea6acccb81eb3168f51abb15',
               '03f547eb9868f48f44d59b56067d4ac4740672c3',
               '159a99bbafdd6c01871228113d7042c3f83f360f',
     

In [9]:
repo_path = DATA_DIR / 'interim' / 'repositories'
repo_path.mkdir(exist_ok=True, parents=True)


def get_new_records(repo_url) -> Tuple[List, Dict]:
    error_data = {
        'clone': [],
        'git': [],
        'no_changed_method_gavs': [],
        'no_java_gavs': [],
        'no_source_code': [],
        'is_test': [],
        'traversal_problems': [],
        'merge_commits': [],
        'no_modifications': [],
    }
    new_records = list()
    commit_hashes = repo_patches[repo_url]

    def original_commit_hash(commit_hash: str) -> str:
        return next(x for x in commit_hashes if (x.startswith(commit_hash) or commit_hash.startswith(x)))

    try:
        repo = pydriller.Repository(repo_url, clone_repo_to=repo_path, include_remotes=True, include_deleted_files=True,
                                    include_refs=True)
    except Exception:
        print(f'error cloning {repo_url}')
        error_data['clone'].append(repo_url)
        return [], error_data

    try:
        repo_commits = [c for c in repo.traverse_commits()]
    except git.exc.CommandError as e:
        print(f'git cmd error ({repo_url}): [{type(e)}: {e}]')
        error_data['git'].append(repo_url)
        return [], error_data
    except Exception as e:
        print(f'error traversing commits ({repo_url}): [{type(e)}: {e}]')
        error_data['git'].append(repo_url)
        return [], error_data

    for ch in commit_hashes:
        to_be_traversed = list(c for c in repo_commits if c.hash.startswith(ch) or ch.startswith(c.hash))
        if len(to_be_traversed) != 1:
            error_data['traversal_problems'].append((repo_url, ch))
            print(f'there is a traversal problem for {(repo_url, ch, len(to_be_traversed))}')

        for commit in to_be_traversed:
            if len(commit.modified_files) == 0:
                if commit.merge:
                    print(f'merge commit {(repo_url, commit.hash)}')
                    error_data['merge_commits'].append((repo_url, commit.hash))
                else:
                    print(f'commit has no modified files: {(repo_url, commit.hash)}')
                    error_data['no_modifications'].append((repo_url, commit.hash))

                continue

            modified_java_files = list(mf for mf in commit.modified_files if mf.filename.endswith('.java'))
            if len(modified_java_files) == 0:
                error_data['no_java_gavs'].append((repo_url, commit.hash))
                continue

            modified_java_files = list(mf for mf in modified_java_files if mf.source_code_before is not None)
            if len(modified_java_files) == 0:
                error_data['no_source_code'].append((repo_url, commit.hash))
                continue

            modified_java_files = list(mf for mf in modified_java_files if len(mf.changed_methods) > 0)
            if len(modified_java_files) == 0:
                error_data['no_changed_method_gavs'].append((repo_url, commit.hash))
                continue

            modified_java_files = list(
                mf for mf in modified_java_files if len(re.compile(r'[Tt]est').findall(mf.old_path)) == 0)
            if len(modified_java_files) == 0:
                error_data['is_test'].append((repo_url, commit.hash))
                continue

            

            for mf in modified_java_files:
                try:
                    new_row = dict()
                    new_row['repo'] = repo_url
                    new_row['commitHash'] = ch
                    new_row['snykPatchUrl'] = repo_commit_to_url[(repo_url, original_commit_hash(commit.hash))]
                    new_row['commitHash'] = commit.hash
                    new_row['modifiedFilePathBefore'] = mf.old_path
                    new_row['modifiedFilePathAfter'] = mf.new_path
                    new_row['modifiedFileSrcBefore'] = mf.source_code_before
                    new_row['modifiedFileSrcAfter'] = mf.source_code
                    new_row['diffParsedJson'] = json.dumps(mf.diff_parsed)
                    new_row['nloc'] = mf.nloc
                    new_row['changedMethods'] = list(map(lambda m: m.name, mf.changed_methods))

                    new_records.append(new_row)

                except Exception:
                    print(f'error parsing modified files {repo_url}: {ch} ({mf.filename})')
                    continue
    return new_records, error_data

In [10]:
res = []
for key in repo_patches.keys():
    res.append(get_new_records(key))

#with Pool(32) as p:
#    res = p.map(get_new_records, repo_patches.keys())

is here 2https://github.com/pgjdbc/pgjdbc
is here 0
is here 1
is here 6
is here 7
is here 8
is here 9
;
<
;
<
;
;
;
;
;
;
;
;
;
;
;
;
;
[
;
;
;
;
;
[
[
;
;
;
;
;
;
<
;
<
;
;
;
;
;
;
;
;
;
.
;
;
;
;
;
;
;
;
;
[
;
;
;
;
;
[
[
;
;
;
;
;
<
;
;
;
;
<
;
;
;
;
;
[
[
[
[
[
[
;
;
;
;
;
;
[
)
)
)
[
;
;
;
;
;
[
[
[
[
[
[
;
;
;
;
;
;
[
)
)
)
[
;
;
;
;
;
)
.
)
)
,
)
;
)
,
)
;
)
)
;
)
)
;
;
)
)
;
)
;
;
)
.
)
)
,
)
;
)
,
)
;
)
)
;
)
)
;
;
)
)
;
)
;
;
)
;
)
;
;
;
)
;
<
;
;
;
)
<
;
)
;
;
)
;
<
;
;
;
;
;
)
<
;
)
<
[
[
is here 12
is here 10
is here 5
is here 4
is here 6
is here 7
is here 8
is here 9
;
;
;
{
;
;
.
[
;
;
;
{
;
;
.
[
is here 12
is here 10
is here 5
is here 4
is here 6
is here 7
is here 8
is here 9
<
;
;
;
;
<
;
;
;
;
;
[
[
[
[
[
[
;
;
;
;
;
;
[
)
)
)
[
;
;
;
;
;
[
[
[
[
[
[
;
;
;
;
;
;
[
)
)
)
[
;
;
;
;
<
<
;
<
;
;
;
<
<
<
<
;
;
;
;
;
;
;
;
[
;
;
;
;
;
,
,
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
<
;
;
<
;
[
)
<
;
<
[
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
;
<
<
;
<
;
;
;
<
<
<
<
;
;
;


In [11]:
new_df = list(chain(*[x[0] for x in res]))
len(new_df), new_df[0].keys()

(1053,
 dict_keys(['repo', 'commitHash', 'snykPatchUrl', 'modifiedFilePathBefore', 'modifiedFilePathAfter', 'modifiedFileSrcBefore', 'modifiedFileSrcAfter', 'diffParsedJson', 'nloc', 'changedMethods']))

In [12]:
errors = collections.defaultdict(set)
for e in [r[1] for r in res]:
    for k, v in e.items():
        errors[k].update(v)
for k, v in errors.items():
    print(f'{k}: {len(v)}')
errors

clone: 0
git: 0
no_changed_method_gavs: 61
no_java_gavs: 20
no_source_code: 1
is_test: 20
traversal_problems: 7
merge_commits: 20
no_modifications: 0


defaultdict(set,
            {'clone': set(),
             'git': set(),
             'no_changed_method_gavs': {('https://github.com/ESAPI/esapi-java-legacy',
               'a0d67b75593878b1b6e39e2acc1773b3effedb2a'),
              ('https://github.com/FasterXML/jackson-databind',
               '05d7e0e13f43e12db6a51726df12c8b4d8040676'),
              ('https://github.com/FasterXML/jackson-databind',
               '1645efbd392989cf015f459a91c999e59c921b15'),
              ('https://github.com/FasterXML/jackson-databind',
               '191a4cdf87b56d2ddddb77edd895ee756b7f75eb'),
              ('https://github.com/FasterXML/jackson-databind',
               '2235894210c75f624a3d0cd60bfb0434a20a18bf'),
              ('https://github.com/FasterXML/jackson-databind',
               '27b4defc270454dea6842bd9279f17387eceb737'),
              ('https://github.com/FasterXML/jackson-databind',
               '28badf7ef60ac3e7ef151cd8e8ec010b8479226a'),
              ('https://github.com/F

In [13]:
commit_urls = set()
commits = set()
repos = set()
for it in new_df:
    commit_urls.add((it['repo'], it['commitHash']))
    repos.add(it['repo'])
    commits.add(it['commitHash'])
len(repos), len(commit_urls)  # 87, 266 commits

(94, 351)

In [14]:
res_df = pd.DataFrame(data=new_df)
res_df  # 1050 files

,repo,commitHash,snykPatchUrl,modifiedFilePathBefore,modifiedFilePathAfter,modifiedFileSrcBefore,modifiedFileSrcAfter,diffParsedJson,nloc,changedMethods
0,https://github.com/pgjdbc/pgjdbc,f6d47034a4ce292e1a659fa00963f6f713117064,https://github.com/pgjdbc/pgjdbc/commit/f6d470...,pgjdbc/src/main/java/org/postgresql/Driver.java,pgjdbc/src/main/java/org/postgresql/Driver.java,"/*\n * Copyright (c) 2003, PostgreSQL Global D...","/*\n * Copyright (c) 2003, PostgreSQL Global D...","{""added"": [[287, "" * this is an empty metho...",453.0,"[Driver::connect, Driver::setupLoggerFromPrope..."
1,https://github.com/pgjdbc/pgjdbc,9008dc9aade6dbfe4efafcd6872ebc55f4699cf5,https://github.com/pgjdbc/pgjdbc/commit/9008dc...,pgjdbc/src/main/java/org/postgresql/util/Strea...,pgjdbc/src/main/java/org/postgresql/util/Strea...,"/*\n * Copyright (c) 2004, PostgreSQL Global D...","/*\n * Copyright (c) 2004, PostgreSQL Global D...","{""added"": [[20, ""import java.nio.file.Files;""]...",150.0,[StreamWrapper::StreamWrapper]
2,https://github.com/pgjdbc/pgjdbc,14b62aca4764d496813f55a43d050b017e01eb65,https://github.com/pgjdbc/pgjdbc/commit/14b62a...,pgjdbc/src/main/java/org/postgresql/jdbc/PgCon...,pgjdbc/src/main/java/org/postgresql/jdbc/PgCon...,"/*\n * Copyright (c) 2004, PostgreSQL Global D...","/*\n * Copyright (c) 2004, PostgreSQL Global D...","{""added"": [[40, ""import org.postgresql.xml.Def...",1397.0,[ReadOnlyBehavior::PgConnection]
3,https://github.com/pgjdbc/pgjdbc,14b62aca4764d496813f55a43d050b017e01eb65,https://github.com/pgjdbc/pgjdbc/commit/14b62a...,pgjdbc/src/main/java/org/postgresql/jdbc/PgSQL...,pgjdbc/src/main/java/org/postgresql/jdbc/PgSQL...,"/*\n * Copyright (c) 2004, PostgreSQL Global D...","/*\n * Copyright (c) 2004, PostgreSQL Global D...","{""added"": [[12, ""import org.postgresql.xml.Def...",250.0,"[equals::return, NonPrintingErrorHandler::erro..."
4,https://github.com/pgjdbc/pgjdbc,0afaa71d5254804f8bab7caab96735c5f7d11aa1,https://github.com/pgjdbc/pgjdbc/commit/0afaa7...,pgjdbc/src/main/java/org/postgresql/jdbc/PgRes...,pgjdbc/src/main/java/org/postgresql/jdbc/PgRes...,"/*\n * Copyright (c) 2004, PostgreSQL Global D...","/*\n * Copyright (c) 2004, PostgreSQL Global D...","{""added"": [[1421, "" Utils.escapeIdentifie...",3106.0,[PgResultSet::refreshRow]
...,...,...,...,...,...,...,...,...,...,...
1048,https://github.com/xerial/snappy-java,d0042551e4a3509a725038eb9b2ad1f683674d94,https://github.com/xerial/snappy-java/commit/d...,src/main/java/org/xerial/snappy/Snappy.java,src/main/java/org/xerial/snappy/Snappy.java,/*--------------------------------------------...,/*--------------------------------------------...,"{""added"": [[172, "" int byteSize = input...",414.0,"[Snappy::compress, Snappy::compress, Snappy::c..."
1049,https://github.com/xerial/snappy-java,820e2e074c58748b41dbd547f4edba9e108ad905,https://github.com/xerial/snappy-java/commit/8...,src/main/java/org/xerial/snappy/BitShuffle.java,src/main/java/org/xerial/snappy/BitShuffle.java,/*--------------------------------------------...,/*--------------------------------------------...,"{""added"": [[94, "" if (input.length * 2 ...",132.0,"[BitShuffle::shuffle, BitShuffle::shuffle, Bit..."
1050,https://github.com/xerial/snappy-java,3bf67857fcf70d9eea56eed4af7c925671e8eaea,https://github.com/xerial/snappy-java/commit/3...,src/main/java/org/xerial/snappy/SnappyInputStr...,src/main/java/org/xerial/snappy/SnappyInputStr...,/*--------------------------------------------...,/*--------------------------------------------...,"{""added"": [[420, "" // chunkSize is nega...",287.0,[SnappyInputStream::hasNextChunk]
1051,https://github.com/SvenEwald/xmlbeam,f8e943f44961c14cf1316deb56280f7878702ee1,https://github.com/SvenEwald/xmlbeam/commit/f8...,src/main/java/org/xmlbeam/config/DefaultXMLFac...,src/main/java/org/xmlbeam/config/DefaultXMLFac...,/**\n * Copyright 2012 Sven Ewald\n *\n * Li...,/**\n * Copyright 2012 Sven Ewald\n *\n * Li...,"{""added"": [[107, "" private static final Str.

In [ ]:
db.patchCommitsLibio.insert_many(new_df)